In [1]:
import selectorlib
from web_scrape.utilities import getHtml
from web_scrape.laptop import getModelNumber
from web_scrape.specifications import get_specifications

In [2]:
flipkart_details = '''
    title:
        css: 'span.B_NuCI'
        type: Text
    rating:
        css: 'div._3LWZlK'
        type: Text
    n_ratings: 
        css: 'span._2_R_DZ'
        type: Text
'''
amazon_details = '''
    title:
        css: 'span.B_NuCI'
        type: Text
    rating:
        css: 'div.centerColAlign a.a-popover-trigger span.a-size-base'
        type: Text
    n_ratings: 
        css: 'div.centerColAlign span.a-declarative a.a-link-normal span.a-size-base'
        type: Text
'''
flipkartProductExtractor =  selectorlib.Extractor.from_yaml_file('web_scrape/selectors/flipkartProductSelector.yml')
flipkartProductExtractorAlternative =  selectorlib.Extractor.from_yaml_file('web_scrape/selectors/flipkartProduct_Alternate.yml')
amazonProductExtractor = selectorlib.Extractor.from_yaml_file('web_scrape/selectors/amazonProductSelector.yml')
flipkartTitleSelector = selectorlib.Extractor.from_yaml_string(flipkart_details)
amazonDetailsSelector = selectorlib.Extractor.from_yaml_string(amazon_details)

In [3]:
def get_average_rating(a, f):
            if a and a['price']:
                a['price'] = int(a['price'].replace("₹", '').replace(',', ''))
            try:
                f_n_ratings = float(f['n_ratings'].split()[0].replace(",", '').replace("(", "").replace(")", ""))
            except:
                f_n_ratings = 0
            try:
                a_n_ratings = float(a['n_ratings'].split()[0].replace(",", ''))
            except: 
                a_n_ratings = 0
            try: 
                f_rating = float(f['rating'])
            except:
                f_rating = 0
            try: 
                
                a_rating = float(a['rating'].split()[0])
            except:
                a_rating = 0
            flag = True # Product contains enough reviews, So it can be added
            if f_n_ratings <10 and a_n_ratings < 10:
                 flag = False # Product Should not be added.
            if not flag:
                 return  {"f": 0, "a": 0, "avg": 0}, False
            avgRating = ( f_n_ratings *  f_rating + a_n_ratings * a_rating) / ( f_n_ratings +  a_n_ratings )


            return {"f": f_rating, "a": a_rating, "avg": avgRating}, flag

In [4]:
def unique_flipkart_product(productsFlipkart):
    new_product_list = []
    reference = []
    for i in range(len(productsFlipkart)):
        if productsFlipkart[i]['ad'] == None:
            cur = productsFlipkart[i]['title'].split('(')[0]
            if cur not in reference:
                reference.append(cur)
                new_product_list.append(productsFlipkart[i])
    return new_product_list

In [5]:
def get_flipkart_products(search_string):
    productsFlipkart = []
    for i in range(1,2):
        flipkartQueryUrl ="https://www.flipkart.com/search?q=" + search_string.replace(" ", "%20")+"&page="+str(i)
        htmlText = getHtml(flipkartQueryUrl)
        try: 
            productsFlipkart += flipkartProductExtractor.extract(htmlText,base_url="https://flipkart.com")["products"][0:11]
        except:
            productsFlipkart += flipkartProductExtractorAlternative.extract(htmlText,base_url="https://flipkart.com")["products"][0:11]
            
    for product in productsFlipkart:
        product['price'] = int(product['price'].replace("₹", '').replace(',', ''))
        productHTML = getHtml(product["url"])
        res = flipkartTitleSelector.extract(productHTML)
        while res['title'] == None and res['rating'] == None  and res['n_ratings'] == None:
            productHTML = getHtml(product["url"])
            res = flipkartTitleSelector.extract(productHTML)
        product['specifications'] = get_specifications(product['url'], search_string.split()[0])
        product["title"] = res['title']
        product['rating'] = res['rating']
        product['n_ratings'] = res['n_ratings']
            
    return unique_flipkart_product(productsFlipkart)

In [6]:
def get_amazon_price_by_fproducts_list(productsFlipkart, type):
        productsAmazon = []
        delete_flipkart_items = []
        for index, product in enumerate(productsFlipkart):
            brand = product['title'].split()[0].lower()
            part = None
            if brand == 'realme' and type == 'laptop':
                delete_flipkart_items.append(index)
                continue
            if brand == 'lenovo' and type == 'laptop': 
                specifications = get_specifications(product['url'],type)
                if 'Part Number' in specifications:
                    part = specifications['Part Number']
            model, matched = getModelNumber(product["title"], part = part)

            product['model'] = model

            amazonQueryUrl = "https://www.amazon.in/s?k=" + model.replace(" ", "+") + "&rh=n%3A1805560031%2Cp_n_condition-type%3A8609960031"
            html = getHtml(amazonQueryUrl)
            temp = amazonProductExtractor.extract(html, base_url="https://amazon.in")

            i = 1
            while temp['products'] == None:
                i+=1
                print(i, end = " ")
                html = getHtml(amazonQueryUrl, mode =  "" if i%2==0 else 'selenium')
                temp = amazonProductExtractor.extract(html, base_url="https://amazon.in")

            amazonProduct = temp['products'][0]

            model2, matched = getModelNumber(amazonProduct['title'])
            
            is_model_same = model.replace(" ", '') == model2.replace(' ','')

            if (matched and is_model_same) or ((not matched) and type == 'mobile'):
                if amazonProduct['rating'] == None or amazonProduct['n_ratings'] == None:
                    html = getHtml(amazonProduct['url'])
                    res = amazonDetailsSelector.extract(html)
                    amazonProduct['rating'] = res['rating']
                    amazonProduct['n_ratings'] = res['n_ratings']
                productsAmazon.append(amazonProduct)

            else:
                productsAmazon.append(None)

            # if matched:
            #     if is_model_same:
            #         productsAmazon.append(amazonProduct)
            #     else:
            #         productsAmazon.append(None)
            # else:
            #     if type == 'mobile':
            #         productsAmazon.append(amazonProduct)
            #     else:
            #         productsAmazon.append(None)
        print()

        productsFlipkart = [product for index, product in enumerate(productsFlipkart) if index not in delete_flipkart_items]

        return productsFlipkart, productsAmazon

In [7]:
def get_overall_products(productsAmazon, productsFlipkart, type):
        overall = []
        models = []

        for i in range(len(productsAmazon)):
            f = productsFlipkart[i]
            a = productsAmazon[i]
            res, flag = get_average_rating(a,f)

            if not flag:
                continue
            
            temp = {
                'title' : f["title"],
                'flipkartUrl': f['url'],
                'amazonUrl': a['url'] if a else None,
                'flipkartRating': res['f'],
                'amazonRating':res['a'] if a else None,
                'averageRating': res['avg'],
                'amazonPrice': a['price'] if a else None,
                'flipkartPrice': f['price'],
                "model":f['model'] if 'model' in f else None,
                'specifications': f['specifications'] if 'specifications' in f else get_specifications(f['url'], type)
            }

            models.append(f['model'])
            f = True
            if['model'] not in models:
                overall.append(temp)

        overall.sort(key=lambda x: x["averageRating"], reverse=True)
        return overall  

In [8]:
def get_search_string(product_i):
    search_string = product_i['type'] + " "
    if 'processor' in product_i['specifications'] :
        search_string +=  product_i['specifications']['processor']
    if "graphics" in product_i['specifications']:
        search_string += " " +  product_i['specifications']['graphics']
    if "RAM" in product_i['specifications']:
        search_string += " " +  product_i['specifications']['RAM']
    if "storage" in product_i['specifications'] and product_i['type'] != 'mobile':
        search_string += " " +  product_i['specifications']['storage']
    if "battery" in product_i['specifications']:
        search_string += " " +  product_i['specifications']['battery']
    if "camera" in product_i['specifications']:
        search_string += " " +  product_i['specifications']['camera']

    return search_string

In [9]:
def get_competitor_products(product_i):
    search_string = get_search_string(product_i)
    
    try:
        productsFlipkart = get_flipkart_products(search_string)
        productsFlipkart, productsAmazon = get_amazon_price_by_fproducts_list(productsFlipkart, product_i['type'])

        overall = get_overall_products(productsAmazon, productsFlipkart, product_i['type'])

        return overall
    except Exception as e:
        with open('exception2.html', 'w') as w:
            w.write(e)
        raise e

In [10]:
data = '''
{
  "_id": {
    "$oid": "6468530f7b6a63fbd9bfe976"
  },
  "name": "Redmi Note 12 Pro 5G",
  "price": 24999,
  "min_price": 23000,
  "max_price": 27500,
  "urls": {
    "flipkartURL": "https://www.flipkart.com/redmi-note-12-pro-5g-onyx-black-128-gb/p/itmbc9fd7adaa32a?pid=MOBGH2UVMHEPGSHM&lid=LSTMOBGH2UVMHEPGSHMZTFAGO&marketplace=FLIPKART&q=redmi+note+12+pro+5g&store=tyy%2F4io&srno=s_1_1&otracker=AS_QueryStore_OrganicAutoSuggest_1_17_na_na_ps&otracker1=AS_QueryStore_OrganicAutoSuggest_1_17_na_na_ps&fm=organic&iid=bca21309-45e0-4508-a6b3-ed22656b404f.MOBGH2UVMHEPGSHM.SEARCH&ppt=hp&ppn=homepage&ssid=328c9au5rk0000001684558548870&qH=51d2caee33332827",
    "amazonURL": "https://www.amazon.in/Redmi-12-Pro-5G-Glacier/dp/B0BRVFZSGV/ref=sr_1_2?crid=3RT9NJ82YM9UK&keywords=REDMI+Note+12+Pro+5G&qid=1684558564&sprefix=redmi%2520note%252012%2520pro%25205g%2520%2Caps%2C262&sr=8-2"
  },
  "type": "mobile",
  "sales": [],
  "scores": [],
  "prices": [
    24999
  ],
  "recommended_price": null,
  "specifications": {
    "battery": "5000 mAh",
    "processor": "Mediatek Dimensity 1080 Octa Core 2.6 GHz",
    "camera": "50MP (OIS) + 8MP + 2MP",
    "RAM": "6 GB",
    "storage": "128 GB",
    "display": "16.94 cm (6.67 inch)2400 x 1080 Pixels",
    "display_type": "Full HD+ AMOLED Display",
    "price": 24999,
    "model": "Note 12 Pro 5G"
  },
  "status": 1
}
'''
import json

data = json.loads(data)

In [11]:
f = get_competitor_products(data)

In [12]:
f

[{'title': 'realme C55 (Sunshower, 128 GB)\xa0\xa0(8 GB RAM)',
  'flipkartUrl': 'https://flipkart.com/realme-c55-sunshower-128-gb/p/itm054283d14c56e?pid=MOBGNBYJUA2G5HK4&lid=LSTMOBGNBYJUA2G5HK40FPVVJ&marketplace=FLIPKART&q=mobile+Mediatek+Dimensity+1080+Octa+Core+2.6+GHz+6+GB+5000+mAh+50MP+%28OIS%29+++8MP+++2MP&store=tyy%2F4io&spotlightTagId=BestsellerId_tyy%2F4io&srno=s_1_2&otracker=search&fm=organic&iid=6acaa397-19d2-491e-b1b0-3fd1714a4490.MOBGNBYJUA2G5HK4.SEARCH&ppt=sp&ppn=sp&ssid=4agewup6ftcbp1q81684560206124&qH=161c53a7077fdc6b',
  'amazonUrl': 'https://amazon.in/realme-C55-Sunshower-128GB-Storage/dp/B0BZDPDLGS/ref=sr_1_1?keywords=realme+C55+%28Sunshower%2C+128+GB%29++%288+GB+RAM%29&qid=1684560240&refinements=p_n_condition-type%3A8609960031&s=electronics&sr=1-1',
  'flipkartRating': 4.4,
  'amazonRating': 3.0,
  'averageRating': 4.399785670545009,
  'amazonPrice': None,
  'flipkartPrice': 13999,
  'model': 'realme C55 (Sunshower, 128 GB)\xa0\xa0(8 GB RAM)',
  'specifications': {'b

In [13]:
from database.mongodb import db

In [14]:
db['competitive_products'].update_one ({ 'product_id': '6468530f7b6a63fbd9bfe976'} , {"$set": { "products": f }} )